In [2]:
import gzip
import json
import os
import io
import csv #to read in and split up all the content from the dataset input file
import numpy as np #import numpy using np as an alias
import pandas as pd #import pandas using pd as an alias
import matplotlib.pyplot as plt #import matplotlib.pyplot using plt as an alias
import seaborn as sb #import seaborn using sns as an alias
import sys
import codecs
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #import the encoder classes

In [31]:
uniqueCards = ['Defend', 'Neutralize', 'Strike', 'Survivor', 'Acrobatics', 'Backflip', 'Bane', 
               'Blade Dance', 'Cloak And Dagger', 'Dagger Spray', 'Dagger Throw', 'Deadly Poison', 
               'Deflect', 'Dodge and Roll', 'Flying Knee', 'Outmaneuver', 'PiercingWail', 
               'Poisoned Stab', 'Prepared', 'Quick Slash', 'Slice', 'Sneaky Strike', 'Sucker Punch', 
               'Accuracy', 'All Out Attack', 'Backstab', 'Blur', 'Bouncing Flask', 
               'Calculated Gamble', 'Caltrops', 'Catalyst', 'Choke', 'Concentrate', 
               'Crippling Cloud', 'Dash', 'Distraction', 'Endless Agony', 'Escape Plan', 
               'Eviscerate', 'Expertise', 'Finisher', 'Flechettes', 'Footwork', 'Heel Hook', 
               'Infinite Blades', 'Leg Sweep', 'Masterful Stab', 'Noxious Fumes', 'Predator', 
               'Reflex', 'Riddle With Holes', 'Setup', 'Skewer', 'Tactician', 'Terror', 
               'Well Laid Plans', 'A Thousand Cuts', 'Adrenaline', 'After Image', 'Alchemize', 
               'Bullet Time', 'Burst', 'Corpse Explosion', 'Die Die Die', 'Doppelganger', 'Envenom', 
               'Glass Knife', 'Grand Finale', 'Malaise', 'Nightmare', 'Phantasmal Killer', 
               'Storm of Steel', 'Tools of the Trade', 'Unload', 'Wraith Form', 'Bandage Up',
               'Blind', 'Dark Shackles', 'Deep Breath', 'Discovery', 'Dramatic Entrance',
               'Enlightenment', 'Finesse', 'Flash of Steel', 'Forethought', 'Good Instincts',
               'Impatience', 'Jack Of All Trades', 'Madness', 'Mind Blast', 'Panacea',
               'PanicButton', 'Purity', 'Swift Strike', 'Trip', 'Apotheosis', 'Chrysalis',
               'HandOfGreed', 'Magnetism', 'Master of Strategy', 'Mayhem', 'Metamorphosis',
               'Panache', 'Sadistic Nature', 'Secret Technique', 'Secret Weapon', 'The Bomb',
               'Thinking Ahead', 'Transmutation', 'Violence', 'Apparition', 'Beta', 'Bite',
               'Expunger', 'Insight', 'J.A.X.', 'Miracle', 'Omega', 'RitualDagger', 'Safety',
               'Shiv', 'Smite', 'Through Violence', 'Clumsy', 'Decay', 'Doubt',
               'Injury', 'Normality', 'Pain', 'Parasite', 'Regret',
               'Shame', 'Writhe', 'Pride', 'Venomology', 'Underhanded Strike', 'Crippling Poison',
               'Night Terror', 'CurseOfTheBell', 'AscendersBane', 'Necronomicurse']

In [32]:
#read in the csv with the cleaned file
draftDfv4 = pd.read_csv('draftDfv3.csv', delimiter=',')

In [33]:
#Delete all rows with 'ghostly' card in master_deck
indicesToDelete = []
index = 0
for deck in draftDfv4['master_deck']:
    cardList = deck.split(',')
    for card in cardList:
        if "+" in card:
            card = card.split("+")[0]
        if not(card in uniqueCards):
            indicesToDelete.append(index)
    index += 1
print(f"Before drop: {len(draftDfv4)}")
draftDfv5 = draftDfv4.drop(indicesToDelete, axis=0)
print(f"After drop: {len(draftDfv5)}")
draftDfv5 = draftDfv5.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

Before drop: 187282
After drop: 181337


In [34]:
upgradedCards = []
for card in uniqueCards:
    upgrade1 = card + "+1"
    upgrade2 = card + "+2"
    upgrade3 = card + "+3"
    upgradedCards.append(upgrade1)
    upgradedCards.append(upgrade2)
    upgradedCards.append(upgrade3)
allPossibleCards = upgradedCards + uniqueCards

In [35]:
#create dictionary from allPossibleCards and initialize to 0.
#This dict will be used to initialize 
type_list = []
for card in allPossibleCards:
    type_list.append('int64')
columnsByType = dict(zip(allPossibleCards, type_list))

In [36]:
#Create empty df and initialize column names
newDf1 = pd.DataFrame(columns = allPossibleCards)

In [37]:
#set types for df columns
newDf1 = newDf1.astype(columnsByType, copy=True, errors='raise')

In [39]:
oneHotEncodedDfs = []
for num in range(0, 19):
    oneHotEncodedDfs.append(newDf1.copy(deep=True))

In [40]:
#Split training data into 19 excel spreadsheets, 10000 rows each
lowerLimit = 0
cardCountKeyValue = dict(zip(newDf1.columns, [0] * len(newDf1.columns)))

for oneHotEncodedDf in oneHotEncodedDfs:
    if lowerLimit == 180000:
        for card in draftDfv5['master_deck'][180000:]:
            cardList = card.split(',')
            for card1 in cardList:
                cardCountKeyValue[card1] += 1
            oneHotEncodedDf.loc[len(oneHotEncodedDf.index)] = list(cardCountKeyValue.values())
            cardCountKeyValue = dict(zip(oneHotEncodedDf.columns, [0] * len(oneHotEncodedDf.columns)))
    else:
        for card in draftDfv5['master_deck'][lowerLimit:lowerLimit+10000]:
            cardList = card.split(',')
            for card1 in cardList:
                cardCountKeyValue[card1] += 1
            oneHotEncodedDf.loc[len(oneHotEncodedDf.index)] = list(cardCountKeyValue.values())
            cardCountKeyValue = dict(zip(oneHotEncodedDf.columns, [0] * len(oneHotEncodedDf.columns)))
    oneHotEncodedDf.to_csv('D:\STS Master Deck - OneHot Encoded\DF' + str(lowerLimit) + '.csv')
    lowerLimit += 10000
    print(str(lowerLimit) + " done.")

10000 done.
20000 done.
30000 done.
40000 done.
50000 done.
60000 done.
70000 done.
80000 done.
90000 done.
100000 done.
110000 done.
120000 done.
130000 done.
140000 done.
150000 done.
160000 done.
170000 done.
180000 done.
190000 done.


In [50]:
draftDfv5.columns

Index(['master_deck', 'current_hp_per_floor', 'victory', 'max_hp_per_floor',
       'ascension_level'],
      dtype='object')

In [52]:
columns = list(draftDfv5.columns)
columns.remove('master_deck')
columns

['current_hp_per_floor', 'victory', 'max_hp_per_floor', 'ascension_level']

In [57]:
def addColumnsToDf(dfExtension):
    #pull 10k rows from the rest of draftDfv5's columns and add them to the csv holding the 
    #data for the master_deck
    columns = list(draftDfv5.columns)
    columns.remove('master_deck')
    lowerLimit = dfExtension
    
    df = pd.read_csv('D:\STS Master Deck - With Max Floor\DF'+ str(dfExtension) + '.csv', delimiter=',', index_col=False)
    for columnName in columns:
        #if the column name is current or max hp
        #clean the data
        columnValuesToAdd = []
        requiresCleaning = False
        if (columnName != 'victory' and columnName != 'ascension_level'):
            requiresCleaning = True
        if lowerLimit == 180000:
            break
            for row in draftDfv5[columnName][180000:]:
                if (requiresCleaning):
                    row = row.strip('][').replace(' ', '').split(',')
                    columnValuesToAdd.append(row[32])

        else:
            for row in draftDfv5[columnName][lowerLimit:lowerLimit + 10000]:
                if (requiresCleaning):
                    row = row.strip('][').replace(' ', '').split(',')
                    columnValuesToAdd.append(row[32])
                else:
                    columnValuesToAdd.append(row)
        df.insert(0, columnName, columnValuesToAdd, True)
    df.to_csv('D:\STS Master Deck - With Max Floor\DF' + str(lowerLimit) + '.csv', index_col=False)
    print(str(dfExtension) + " done.")
        
for dfExtension in range(0, 190000, 10000):
    addColumnsToDf(dfExtension)
        

0 done.
10000 done.
20000 done.
30000 done.
40000 done.
50000 done.
60000 done.
70000 done.
80000 done.
90000 done.
100000 done.
110000 done.
120000 done.
130000 done.
140000 done.
150000 done.
160000 done.
170000 done.
180000 done.


In [17]:
def removeRowsWithZeroCurrentHealth(dfExtension):
    indicesToRemove = []
    df = pd.read_csv('D:\STS Master Deck - OneHot Encoded\DF'+ str(dfExtension) + '.csv', delimiter=',', index_col=False)
    row = 0
    
    for currentHealth in df['current_hp_per_floor']:
        if currentHealth == 0:
            indicesToRemove.append(row)
        row += 1
    df = df.drop(indicesToRemove, axis=0)
    df.to_csv('D:\STS Master Deck - OneHot Encoded\DF' + str(dfExtension) + '.csv', index=False)

def removeUnwantedColumn(dfExtension):
    df = pd.read_csv('D:\STS Master Deck - OneHot Encoded\DF'+ str(dfExtension) + '.csv', delimiter=',', index_col=False)
    df = df.drop('Unnamed: 0', axis=1)
    df.to_csv('D:\STS Master Deck - OneHot Encoded\DF' + str(dfExtension) + '.csv', index=False)

for dfExtension in range(0, 180000, 10000):
    removeUnwantedColumn(dfExtension)